# Testing out Local LLMs

In [1]:
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [2]:
import numpy as np

In [3]:
# Loading the model
# llm = ChatOllama(model="llama3.2:1b", temperature=0)
llm = ChatOllama(model="llama3.2", temperature=0)

In [6]:
initialSystemMessage = '''You are an excellent virtual assistant to help with payment of bills.
You should follow the following steps strictly to pay the bills:
1. Ask the user the bill number
2. Ask the user if you should fetch the bill details.
3. Inform the user the bill details once fetched.
4. Ask the user if you should pay the bill.
5. Is the user agrees, pay the bill.

You should never user external knowledge, assumptions or information beyond what is explicitly shared or recieved.
'''


messages = [
    ("system",initialSystemMessage),
    ("human", "Pay my bill")
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="To proceed with paying your bill, I need to know a few details first.\n\nCan you please tell me the number of the bill you'd like to pay?", additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-02-12T22:14:36.447587Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1133021959, 'load_duration': 29848625, 'prompt_eval_count': 127, 'prompt_eval_duration': 201000000, 'eval_count': 33, 'eval_duration': 900000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-d4d26844-08a5-4aad-9a64-013bf559dd9c-0', usage_metadata={'input_tokens': 127, 'output_tokens': 33, 'total_tokens': 160})

## Testing Tools

### Calculator Tools

In [4]:
@tool
def add(a: int, b: int) -> int:
    """Add (+) two numbers"""
    return a+b
    
@tool
def sub(a: int, b: int) -> int:
    """Subtract (-) two numbers"""
    return a-b
    
@tool
def mul(a: int, b: int) -> int:
    """Multiply (*, x) two numbers"""
    return a*b
    
@tool
def div(a: int, b: int) -> int:
    """Divide (/) two numbers"""
    return a/b

@tool
def fetchUserDets(name: str) -> str:
    """Function to fetch user details. Input is name of the user. Output relevant details."""
    # hmap = 
    pass



tools = [mul, div, add, sub]
toolsMap = {'add':add, "sub":sub, "mul":mul, "div":div}

llmTools = llm.bind_tools(tools)


In [14]:
query = "Add 3 and 4?"
query = "What is 3 multiplied 12? Also, what is 11 + 49?"

# llmTools.invoke(query).tool_calls

messages = [
    ("system","You are a helpful assistant. be kind"),
    ("human", "Tell me the answer to 3 * 12+8 * 213")
]
ai_msg = llmTools.invoke(messages)
messages.append(ai_msg)
ai_msg

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-02-24T23:26:50.365703Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1046341750, 'load_duration': 33897542, 'prompt_eval_count': 347, 'prompt_eval_duration': 215000000, 'eval_count': 28, 'eval_duration': 795000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-5029179c-cd05-42d2-842b-2a7f364e646f-0', tool_calls=[{'name': 'add', 'args': {'a': '3 * 12', 'b': '8 * 213'}, 'id': '2d90d4f5-9bda-497f-b61b-4514d504997c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 347, 'output_tokens': 28, 'total_tokens': 375})

In [15]:
ai_msg.tool_calls

[{'name': 'add',
  'args': {'a': '3 * 12', 'b': '8 * 213'},
  'id': '2d90d4f5-9bda-497f-b61b-4514d504997c',
  'type': 'tool_call'}]

In [16]:
messages

[('system', 'You are a helpful assistant. be kind'),
 ('human', 'Tell me the answer to 3 * 12+8 * 213'),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-02-24T23:26:50.365703Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1046341750, 'load_duration': 33897542, 'prompt_eval_count': 347, 'prompt_eval_duration': 215000000, 'eval_count': 28, 'eval_duration': 795000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-5029179c-cd05-42d2-842b-2a7f364e646f-0', tool_calls=[{'name': 'add', 'args': {'a': '3 * 12', 'b': '8 * 213'}, 'id': '2d90d4f5-9bda-497f-b61b-4514d504997c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 347, 'output_tokens': 28, 'total_tokens': 375})]

In [17]:
for toolCall in ai_msg.tool_calls:
    toolSelected = toolsMap[toolCall['name']]
    toolMsg = toolSelected.invoke(toolCall)
    messages.append(toolMsg)
    
messages

ValidationError: 2 validation errors for add
a
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='3 * 12', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/int_parsing
b
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='8 * 213', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/int_parsing

In [11]:
toolCall

{'name': 'sub',
 'args': {'a': 11, 'b': 49},
 'id': '2067c8e1-3fa8-435b-beba-7b95f6cfca06',
 'type': 'tool_call'}

In [8]:
ai_msg = llmTools.invoke(messages)

In [9]:
print(ai_msg.content)

The answer to 3 * 12 is 36.

The answer to 11 - 49 is -38.


In [10]:
ai_msg

AIMessage(content='The answer to 3 * 12 is 36.\n\nThe answer to 11 - 49 is -38.', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-02-24T23:24:26.259054Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1108464084, 'load_duration': 31862125, 'prompt_eval_count': 142, 'prompt_eval_duration': 388000000, 'eval_count': 25, 'eval_duration': 685000000, 'message': Message(role='assistant', content='The answer to 3 * 12 is 36.\n\nThe answer to 11 - 49 is -38.', images=None, tool_calls=None)}, id='run-a1d082b4-c1cd-4821-b174-4d7676facfed-0', usage_metadata={'input_tokens': 142, 'output_tokens': 25, 'total_tokens': 167})

In [15]:
print(ai_msg.content)

The answer to 3 * 12 is 36.

The answer to 11 - 49 is -38.


---

In [1]:
!which python

/Users/asutoshdalei/Desktop/Work/solar/bin/python


In [2]:
import sys
sys.executable

'/Users/asutoshdalei/Desktop/Work/solar/bin/python3.11'

In [5]:
initialSystemMessage = '''You are an excellent virtual assistant to help with payment of bills.
You should follow the following steps strictly to pay the bills:
1. Ask the user the bill number
2. Ask the user if you should fetch the bill details.
3. Inform the user the bill details once fetched.
4. Ask the user if you should pay the bill.
5. Is the user agrees, pay the bill.

You should never user external knowledge, assumptions or information beyond what is explicitly shared or recieved.
'''


In [6]:
print(initialSystemMessage)

You are an excellent virtual assistant to help with payment of bills.
You should follow the following steps strictly to pay the bills:
1. Ask the user the bill number
2. Ask the user if you should fetch the bill details.
3. Inform the user the bill details once fetched.
4. Ask the user if you should pay the bill.
5. Is the user agrees, pay the bill.

You should never user external knowledge, assumptions or information beyond what is explicitly shared or recieved.

